# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 14 2022 11:46AM,241874,10,PRF-29831,Bio-PRF,Released
1,Jul 14 2022 11:46AM,241874,10,PRF-30398-BO,Bio-PRF,Released
2,Jul 14 2022 11:46AM,241874,10,PRF-30637,Bio-PRF,Released
3,Jul 14 2022 11:46AM,241874,10,PRF-30576-BO,Bio-PRF,Released
4,Jul 14 2022 11:46AM,241874,10,PRF-30451-BO,Bio-PRF,Released
5,Jul 14 2022 11:46AM,241874,10,PRF-30653,Bio-PRF,Released
6,Jul 14 2022 11:46AM,241874,10,PRF-29848-BO1,Bio-PRF,Released
7,Jul 14 2022 11:46AM,241874,10,PRF-30794,Bio-PRF,Released
8,Jul 14 2022 11:46AM,241874,10,PRF-28486-BO1,Bio-PRF,Released
9,Jul 14 2022 11:46AM,241874,10,PRF-30776-BO,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,241869,Released,1
39,241871,Released,1
40,241872,Released,1
41,241873,Released,1
42,241874,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241869,NaN,NaN,1.0
241871,NaN,NaN,1.0
241872,NaN,NaN,1.0
241873,NaN,NaN,1.0
241874,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241771,0.0,1.0,0.0
241784,0.0,0.0,1.0
241785,0.0,1.0,0.0
241787,0.0,0.0,1.0
241793,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241771,0,1,0
241784,0,0,1
241785,0,1,0
241787,0,0,1
241793,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241771,0,1,0
1,241784,0,0,1
2,241785,0,1,0
3,241787,0,0,1
4,241793,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241771,,1,
1,241784,,,1
2,241785,,1,
3,241787,,,1
4,241793,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 14 2022 11:46AM,241874,10,Bio-PRF
13,Jul 14 2022 11:38AM,241873,21,"NBTY Global, Inc."
14,Jul 14 2022 11:29AM,241872,21,"NBTY Global, Inc."
15,Jul 14 2022 11:23AM,241871,21,"NBTY Global, Inc."
16,Jul 14 2022 11:19AM,241869,21,"NBTY Global, Inc."
17,Jul 14 2022 11:14AM,241864,10,ISDIN Corporation
18,Jul 14 2022 11:14AM,241866,10,ISDIN Corporation
34,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions
35,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation
37,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 14 2022 11:46AM,241874,10,Bio-PRF,,,13
1,Jul 14 2022 11:38AM,241873,21,"NBTY Global, Inc.",,,1
2,Jul 14 2022 11:29AM,241872,21,"NBTY Global, Inc.",,,1
3,Jul 14 2022 11:23AM,241871,21,"NBTY Global, Inc.",,,1
4,Jul 14 2022 11:19AM,241869,21,"NBTY Global, Inc.",,,1
5,Jul 14 2022 11:14AM,241864,10,ISDIN Corporation,,,1
6,Jul 14 2022 11:14AM,241866,10,ISDIN Corporation,,,16
7,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions,,,1
8,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation,,,2
9,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc.",,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 11:46AM,241874,10,Bio-PRF,13,,
1,Jul 14 2022 11:38AM,241873,21,"NBTY Global, Inc.",1,,
2,Jul 14 2022 11:29AM,241872,21,"NBTY Global, Inc.",1,,
3,Jul 14 2022 11:23AM,241871,21,"NBTY Global, Inc.",1,,
4,Jul 14 2022 11:19AM,241869,21,"NBTY Global, Inc.",1,,
5,Jul 14 2022 11:14AM,241864,10,ISDIN Corporation,1,,
6,Jul 14 2022 11:14AM,241866,10,ISDIN Corporation,16,,
7,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions,1,,
8,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation,2,,
9,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc.",10,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 11:46AM,241874,10,Bio-PRF,13,,
1,Jul 14 2022 11:38AM,241873,21,"NBTY Global, Inc.",1,,
2,Jul 14 2022 11:29AM,241872,21,"NBTY Global, Inc.",1,,
3,Jul 14 2022 11:23AM,241871,21,"NBTY Global, Inc.",1,,
4,Jul 14 2022 11:19AM,241869,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 11:46AM,241874,10,Bio-PRF,13.0,NaN,NaN
1,Jul 14 2022 11:38AM,241873,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jul 14 2022 11:29AM,241872,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jul 14 2022 11:23AM,241871,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jul 14 2022 11:19AM,241869,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 11:46AM,241874,10,Bio-PRF,13.0,0.0,0.0
1,Jul 14 2022 11:38AM,241873,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jul 14 2022 11:29AM,241872,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jul 14 2022 11:23AM,241871,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jul 14 2022 11:19AM,241869,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3144041,163.0,26.0,0.0
12,241771,0.0,1.0,0.0
15,241859,3.0,0.0,0.0
16,1692731,6.0,2.0,0.0
19,967351,30.0,17.0,18.0
20,725481,4.0,25.0,0.0
21,1934698,8.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3144041,163.0,26.0,0.0
1,12,241771,0.0,1.0,0.0
2,15,241859,3.0,0.0,0.0
3,16,1692731,6.0,2.0,0.0
4,19,967351,30.0,17.0,18.0
5,20,725481,4.0,25.0,0.0
6,21,1934698,8.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,163.0,26.0,0.0
1,12,0.0,1.0,0.0
2,15,3.0,0.0,0.0
3,16,6.0,2.0,0.0
4,19,30.0,17.0,18.0
5,20,4.0,25.0,0.0
6,21,8.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,163.0
1,12,Released,0.0
2,15,Released,3.0
3,16,Released,6.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,18.0,0.0,0.0
Executing,26.0,1.0,0.0,2.0,17.0,25.0,0.0
Released,163.0,0.0,3.0,6.0,30.0,4.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,18.0,0.0,0.0
1,Executing,26.0,1.0,0.0,2.0,17.0,25.0,0.0
2,Released,163.0,0.0,3.0,6.0,30.0,4.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,18.0,0.0,0.0
1,Executing,26.0,1.0,0.0,2.0,17.0,25.0,0.0
2,Released,163.0,0.0,3.0,6.0,30.0,4.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()